In [24]:
# import sys
# sys.path.insert(0, '..')
from semantic_mpc_interface import (
    LoadModel,
    get_thermostat_data,
    HPFlexSurvey,
    convert_units,
    SHACLHandler,
    inline_shapes
    # add_connection
)
from buildingmotif.namespaces import BRICK, RDF, S223
from rdflib import URIRef
from buildingmotif import BuildingMOTIF
from buildingmotif.dataclasses import Library
import csv
from pyshacl.rdfutil import clone

# SELECT ONTOLOGY 
ontology = 's223'

# base path
base_path = f'{ontology}-example/bldg1'

# Testing the Model Builder

ModelBuilder has been deprecated in favor of just survey based workflow. May be readded in the future

# Testing the Survey Workflow

In [25]:
# Please disregard excessive outputs (logging and warnings)
# TODO: Figure out how to configure these in building motif 
import logging
logging.disable(logging.CRITICAL)
import warnings
warnings.filterwarnings("ignore")

In [28]:
# Creating survey, allow overwrite if there is something already there
s = HPFlexSurvey(base_path.split('/')[0],base_path.split('/')[1],'.', overwrite=True, ontology=ontology,
                 template_dict = {'zone': 'hvac-zone',
                                  "space": "space",
                                  "hvac": "hp-rtu",
                                  "tstat": "tstat",
                                  "window": "window",
                                  "site": "site" })

# Generating a simple building structure that prefills csv files. 
s.easy_config(zone_space_window_list=[(2,2),(1,2),(1,3)])
# Now check test_site/test_build to look at survey files


removing dependency:  space
removing dependency:  window
removing dependency:  hp-rtu
removing dependency:  tstat
values: dict_values([])
values: dict_values(['area-value'])
values: dict_values(['heating_COP-value', 'cooling_capacity-value', 'cooling_COP-value', 'heating_capacity-value'])
values: dict_values(['resolution-value', 'tolerance-value', 'stage_count-value', 'active-value', 'setpoint_deadband-value'])
values: dict_values(['tilt-value', 'area-value', 'azimuth-value'])
values: dict_values(['timezone-value', 'latitude-value', 'noaastation-value', 'longitude-value'])
{'site_id': 's223-example', 'hvac_type': 'hp-rtu', 'variable_params': {'hvac-zone': {}, 'space': {'area': '<name>-area'}, 'hp-rtu': {'heating_COP': '<name>-heating_COP', 'cooling_capacity': '<name>-cooling_capacity', 'cooling_COP': '<name>-cooling_COP', 'heating_capacity': '<name>-heating_capacity'}, 'tstat': {'setpoint_deadband': '<name>-setpoint_deadband', 'resolution': '<name>-resolution', 'tolerance': '<name>-tol

In [27]:
# Will just fill the columns programmatically for testing, csv should be filled out otherwise
import sys
sys.path.insert(0,'../examples')
from example_prefill_usage import prefill_csv_survey

In [ ]:
# prefill_csv_survey(base_path)

Loading config from: s223-test_site/test_build/config.json
Found CSV files: ['zone', 'site', 'window', 'hvac', 'point_list', 'tstat', 'space']
Filled 10 empty cells in zone.csv
Filled 4 empty cells across 4 columns in site.csv
Filled 21 empty cells across 3 columns in window.csv
Filled 12 empty cells across 4 columns in hvac.csv
No empty cells found in point_list.csv
Filled 15 empty cells across 5 columns in tstat.csv
Filled 4 empty cells across 1 columns in space.csv


In [29]:
# Reading csv
s.read_csv()

Expanded CSV shape: (6, 5)
New columns added: []
Expanded CSV shape: (4, 4)
New columns added: ['area']
Expanded CSV shape: (3, 13)
New columns added: ['heating_COP', 'cooling_capacity', 'cooling_COP', 'heating_capacity']
Expanded CSV shape: (3, 14)
New columns added: ['setpoint_deadband', 'resolution', 'tolerance', 'stage_count', 'active']
Expanded CSV shape: (7, 10)
New columns added: ['tilt', 'azimuth', 'area']
Expanded CSV shape: (1, 9)
New columns added: ['noaastation', 'timezone', 'longitude', 'latitude']


# Testing SHACL Generation and Validation

In [30]:
og = clone.clone_graph(s.graph)
# Create handler
handler = SHACLHandler(ontology=ontology)

# Generate shapes
handler.generate_shapes()

# Save shapes
handler.save_shapes(f'{base_path}/shapes.ttl')
# also save more human readable shapes
inline_shapes(handler.shapes_graph).serialize(f"{base_path}/inlined_shapes.ttl")

# Run inference on model
inferred_graph = handler.infer(s.graph)

inferred_graph.serialize(f'{base_path}/reasoned.ttl', format = 'ttl')
# if not validation_result.valid:
#     print("Validation failed:")
#     print(validation_result.report_string)

<Graph identifier=14cfff1e-cee0-47a5-b4c7-5efb96339316 (<class 'rdflib.graph.Graph'>)>

In [31]:
# lots of new inferred information
(inferred_graph-og).print()

@prefix ns1: <urn:hpflex/shapes#> .

<urn:hpflex/s223-example#hvac_1> a ns1:hp-rtu ;
    ns1:air-connected-to <urn:hpflex/s223-example#zone_1> ;
    ns1:has-point <urn:hpflex/s223-example#hvac_1-cooling_COP>,
        <urn:hpflex/s223-example#hvac_1-cooling_capacity>,
        <urn:hpflex/s223-example#hvac_1-heating_COP>,
        <urn:hpflex/s223-example#hvac_1-heating_capacity> .

<urn:hpflex/s223-example#hvac_2> a ns1:hp-rtu ;
    ns1:air-connected-to <urn:hpflex/s223-example#zone_2> ;
    ns1:has-point <urn:hpflex/s223-example#hvac_2-cooling_COP>,
        <urn:hpflex/s223-example#hvac_2-cooling_capacity>,
        <urn:hpflex/s223-example#hvac_2-heating_COP>,
        <urn:hpflex/s223-example#hvac_2-heating_capacity> .

<urn:hpflex/s223-example#hvac_3> a ns1:hp-rtu ;
    ns1:air-connected-to <urn:hpflex/s223-example#zone_3> ;
    ns1:has-point <urn:hpflex/s223-example#hvac_3-cooling_COP>,
        <urn:hpflex/s223-example#hvac_3-cooling_capacity>,
        <urn:hpflex/s223-example#hvac_3-

In [32]:
sg = handler.shapes_graph

In [33]:
from collections import Counter
import pandas as pd
import yaml
from rdflib import Graph, Literal, URIRef, BNode
def inline_shapes(g: Graph):
    # 1. Get all entities defined in this graph (subject of rdf:type)
    defined_entities = set(s for s, p, o in g.triples((None, RDF.type, None)) if isinstance(s, URIRef))

    # 2. Count object references (excluding literals)
    object_counter = Counter()
    for s, p, o in g:
        if isinstance(o, URIRef):
            object_counter[o] += 1

    # 3. Find single-use, defined entities
    candidates = {uri for uri in defined_entities if object_counter[uri] == 1}

    # 4. Map for URIRef -> BNode replacement
    uri_to_bnode = {}
    new_g = Graph()
    for prefix, ns in g.namespaces():
        new_g.bind(prefix, ns)

    # 5. Build the mapping for nodes to be replaced
    for s, p, o in g:
        if isinstance(o, URIRef) and o in candidates:
            if o not in uri_to_bnode:
                uri_to_bnode[o] = BNode()

    # 6. Rewrite the graph with substitutions
    for s, p, o in g:
        # Replace object if needed
        if isinstance(o, URIRef) and o in uri_to_bnode:
            o = uri_to_bnode[o]
        # Replace subject if needed (very rare, but possible)
        if isinstance(s, URIRef) and s in uri_to_bnode:
            s = uri_to_bnode[s]
        new_g.add((s, p, o))
    return new_g, candidates


In [34]:
sgi, candidates = inline_shapes(sg)

In [35]:
candidates

{rdflib.term.URIRef('urn:hpflex/shapes#active-cooling-stagesAnnotationRule'),
 rdflib.term.URIRef('urn:hpflex/shapes#active-cooling-stages_Aspect-OperatingStatus'),
 rdflib.term.URIRef('urn:hpflex/shapes#active-cooling-stages_Aspect-OperatingStatus-1'),
 rdflib.term.URIRef('urn:hpflex/shapes#active-cooling-stages_Count'),
 rdflib.term.URIRef('urn:hpflex/shapes#active-cooling-stages_Count-1'),
 rdflib.term.URIRef('urn:hpflex/shapes#active-cooling-stages_NUM'),
 rdflib.term.URIRef('urn:hpflex/shapes#active-cooling-stages_NUM-1'),
 rdflib.term.URIRef('urn:hpflex/shapes#active-cooling-stages_Role-Cooling'),
 rdflib.term.URIRef('urn:hpflex/shapes#active-cooling-stages_Role-Cooling-1'),
 rdflib.term.URIRef('urn:hpflex/shapes#active-cooling-stages_hasAspect'),
 rdflib.term.URIRef('urn:hpflex/shapes#active-cooling-stages_hasQuantityKind'),
 rdflib.term.URIRef('urn:hpflex/shapes#active-cooling-stages_hasUnit'),
 rdflib.term.URIRef('urn:hpflex/shapes#active-heating-stagesAnnotationRule'),
 rdfli

# Testing get Metadata

In [36]:
# still working on loader, will clean up class, but functionality about right
loader = LoadModel(f"{base_path}/reasoned.ttl", 
                   template_dict={'sites':'site', 'zones': 'hvac-zone'}, 
                   ontology = ontology)
site_info = loader.get_all_building_objects()

In [37]:
print(site_info)

{'sites': [site(name='urn:hpflex/s223-example#s223-example', timezone=Value(value=US/Pacific, unit='None'), noaastation=Value(value=, unit='None'), longitude=Value(value=-122.254433, unit='http://qudt.org/vocab/unit/Degree'), latitude=Value(value=37.879134, unit='http://qudt.org/vocab/unit/Degree'), )], 'zones': [hvac_zone(name='urn:hpflex/s223-example#zone_3', , windows=1, spaces=1, hp_rtus=1, tstats=1), hvac_zone(name='urn:hpflex/s223-example#zone_2', , windows=1, spaces=1, hp_rtus=1, tstats=1), hvac_zone(name='urn:hpflex/s223-example#zone_1', , windows=2, spaces=2, hp_rtus=1, tstats=1)]}


In [38]:
zone = site_info['zones'][0]
print(zone)

hvac_zone(name='urn:hpflex/s223-example#zone_3', , windows=1, spaces=1, hp_rtus=1, tstats=1)


In [39]:
zone.spaces

[space(name='urn:hpflex/s223-example#space_3_1', area=Value(value=180.0, unit='http://qudt.org/vocab/unit/FT2'), )]

In [40]:
zone.windows[0].area.name

rdflib.term.URIRef('urn:hpflex/s223-example#window_3_3-area')

In [41]:
zone.tstats[0]

tstat(name='urn:hpflex/s223-example#tstat_zone_3', tstat_tolerance=Value(value=0.5, unit='http://qudt.org/vocab/unit/DEG_F'), tstat_stage_count=Value(value=4.0, unit='http://qudt.org/vocab/unit/NUM'), tstat_deadband=Value(value=0.5, unit='http://qudt.org/vocab/unit/DEG_F'), tstat_resolution=None, tstat_active=Value(value=True, unit='http://qudt.org/vocab/unit/NUM'), )

In [42]:
print(zone.tstats[0].resolution)
zone.tstats[0].resolution.convert_to_si()
print(zone.tstats[0].resolution)
print(zone.tstats[0].resolution.is_delta)

AttributeError: 'tstat' object has no attribute 'resolution'

In [ ]:
# optionally just load everything as si 
si_loader = LoadModel(f"{ontology}-test_site/test_build/test_build.ttl", ontology = 'brick', as_si_units=True)
site_info = si_loader.get_all_building_objects()
print(zone.tstats[0].resolution)

Value(value=1.6666666666666667, unit='http://qudt.org/vocab/unit/DEG_C')


In [ ]:
# Getting the thermostat metadata
get_thermostat_data(si_loader)

{'heat_availability': [],
 'cool_availability': [],
 'heat_tolerance': [],
 'cool_tolerance': [],
 'setpoint_deadband': [],
 'active': [],
 'control_group': [],
 'control_type_list': [],
 'floor_area_list': [],
 'floor_area_unit': [],
 'window_area_list': [],
 'window_area_unit': [],
 'azimuth_list': [],
 'azimuth_unit': [],
 'tilt_list': [],
 'tilt_unit': [],
 'zone_ids': [],
 'hvacs': [],
 'setpoint_type': [],
 'fuel_heat_list': [],
 'fuel_cool_list': [],
 'cooling_capacity': [],
 'cooling_capacity_unit': [],
 'heating_capacity': [],
 'heating_capacity_unit': [],
 'cooling_cop': [],
 'heating_cop': [],
 'cooling_electricity': [],
 'heating_electricity': [],
 'resolution': [],
 'temperature_unit': []}

In [ ]:
# Getting the thermostat metadata for 1 zone 
get_thermostat_data(si_loader, ['zone_1','zone_2'])

{'heat_availability': [],
 'cool_availability': [],
 'heat_tolerance': [],
 'cool_tolerance': [],
 'setpoint_deadband': [],
 'active': [],
 'control_group': [],
 'control_type_list': [],
 'floor_area_list': [],
 'floor_area_unit': [],
 'window_area_list': [],
 'window_area_unit': [],
 'azimuth_list': [],
 'azimuth_unit': [],
 'tilt_list': [],
 'tilt_unit': [],
 'zone_ids': [],
 'hvacs': [],
 'setpoint_type': [],
 'fuel_heat_list': [],
 'fuel_cool_list': [],
 'cooling_capacity': [],
 'cooling_capacity_unit': [],
 'heating_capacity': [],
 'heating_capacity_unit': [],
 'cooling_cop': [],
 'heating_cop': [],
 'cooling_electricity': [],
 'heating_electricity': [],
 'resolution': [],
 'temperature_unit': []}

# Testing Unit Conversion

In [ ]:
convert_units(10, 'FT', 'M')

3.048

In [ ]:
convert_units(0, 'DEG_C', 'DEG_F')

31.999999999999886

In [ ]:
convert_units(0, 'DEG_C', 'K')

273.15

In [ ]:
# Planning to implement converting everything to si in loader
# loader.get_all_building_objects

# Testing Grafana Dashboarding 
Needs updates

In [ ]:
# import yaml 
# with open('../development_files/creds.yml') as f:
#     config = yaml.safe_load(f)

# bg = BrickToGrafana(grafana_server=config['grafana_server'], grafana_api_key = config['grafana_api_key'], datasource=config['datasource'], ttl_path = 'test-brick-model.ttl')

In [ ]:
# bg.create_dashboard('AQL from brick')

In [ ]:
# Grafana not set up, upload won't work
# bg.upload_dashboard(message = 'testing upload')